# Докер развертывание

![pics\2025-12-09_16-56-17.png](pics/2025-12-09_16-56-17.png)

## Основные инструкции

FROM — задаёт базовый (родительский) образ

LABEL — описывает метаданные. Например — сведения о том, кто создал и поддерживает образ

ENV — устанавливает постоянные переменные среды

RUN — выполняет команду и создаёт слой образа. Используется для установки в контейнер пакетов

COPY — копирует в контейнер файлы и папки

ADD — копирует файлы и папки в контейнер, может распаковывать локальные .tar-файлы

CMD — описывает команду с аргументами, которую нужно выполнить когда контейнер будет запущен. Аргументы могут быть переопределены при запуске контейнера. В файле может присутствовать лишь одна инструкция CMD!

WORKDIR — задаёт рабочую директорию для следующей инструкции

ARG — задаёт переменные для передачи Docker во время сборки образа

ENTRYPOINT — предоставляет команду с аргументами для вызова во время выполнения контейнера. Аргументы не переопределяются

EXPOSE — указывает на необходимость открыть порт

VOLUME — создаёт точку монтирования для работы с постоянным хранилищем

<span style="color:green">создаем папку для проекта и переходим в нее</span>

$ mkdir titanic && cd titanic


<span style="color:green">создаем виртуальное окружение и активируем</span>

$ python3 -m venv venv && source venv/bin/activate


<span style="color:green">обновляем пакетный менеджер pip:</span>

$ pip install --upgrade pip


<span style="color:green">добавляем в проект четыре файла</span>

$ touch eda.py model.py app.py Dockerfile

# eda.py

```python

imports

if __name__ == '__main__':
        code here
```

# model.py

```python
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split

if __name__ == '__main__':
    data = pd.read_csv('preprocessed_data.csv')

    x = data.drop(['Survived'], axis=1)
    y = data.Survived

    loss_function = 'Logloss'
    learning_rate = None
    iterations = 1000
    custom_loss = 'Accuracy'

    model = CatBoostClassifier(iterations=iterations,
                               custom_loss=[custom_loss],
                               loss_function=loss_function,
                               learning_rate=learning_rate
                               )

    x_train, x_test, y_train, y_test = train_test_split(
        x,
        y,
        train_size=.85,
        random_state=42
    )
    is_categorical_feature = np.where(x_train.dtypes != float)[0]

    model.fit(x_train,
              y_train,
              cat_features=is_categorical_feature,
              eval_set=(x_test, y_test)
              )

    cv(Pool(x,
            y,
            cat_features=is_categorical_feature
            ),
       model.get_params(),
       fold_count=5
       )

    model.save_model('trained_model.cbm')



```

# app.py

``` python

from flask import Flask, request, jsonify
import pandas as pd
from catboost import CatBoostClassifier

app = Flask(__name__)
model = CatBoostClassifier()
model.load_model('trained_model.cbm')


@app.route('/predict', methods=['POST'])
def predict():
    content = request.json
    df = pd.DataFrame(content, index=[0])
    df = df.drop(['PassengerId'], axis=1)
    print(df)
    result = model.predict_proba(df)
    result = {'Survived_Probability': result[0, 1]}
    return jsonify(result)


if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0')

```

# Dockerfile

```
# Use an official Python runtime as a parent image
FROM python:3.11-slim


# Set the working directory in the container
WORKDIR /usr/src/app

# Copy the current directory contents into the container at /usr/src/app
COPY . .

# Install any needed packages specified in requirements.txt 
RUN pip install --no-cache-dir -r requirements.txt

# Make port 5000 available to the world outside this container (for web apps - 80 or 8080)
EXPOSE 5000

# Define environment variable (optional)
ENV NAME World

# Run app.py when the container launches
CMD ["python", "./app.py"]

```

# Сборка и запуск

```
обираем образ
$ docker build -t mlmodel:latest .
 

Запускаем контейнер
$ docker run -p 5000:5000 mlmodel:latest &
 

Тестируем что получилось
$ diff <(curl -d '{"PassengerId": 762, "Pclass":3, "Name": "Nirva, Mr. Lisakki Antino Aijo", "Sex": "female", "Age": 34, "SibSp": 4, "Parch": 3, "Ticket": "a", "Fare": 1.0, "Cabin": "A", "Embarked": "A"}' -H 'Content-Type: application/json' http://localhost:5000/predict  | tr -d "\n ") <(echo '{"Survived_Probability":0.6485056503550757}' | tr -d "\n")


```

# Публикация в Docker Hub

```
Переходим на Docker Hub, создаем новый Docker ID и выбираем personal subscription
 
Выбираем Create Repository и создаем репозиторий (задаем ему имя mlmodel)
 
Добавляем метку latest к нашему образу
$ docker tag mlmodel:latest <ваш логин на Docker Hub>/mlmodel:latest
 
Авторизуемся локально при помощи команды docker login
$ docker login
 
Загружаем образ на Docker hub командой docker push <имя образа>
$ docker push <ваш логин на Docker Hub>/mlmodel:latest
```

# Docker deployment

экспортировать дашборд в yaml формат

вынести запуск ExplainerDashboard в отдельный файл app.py

написать Dockerfile

# Yaml

YAML — это простой формат для сериализации данных, который позволяет хранить сложноорганизованные объекты в компактном и читаемом формате. Наряду с JSON и XML является популярным выбором при конфигурации приложений и сохранении объектов.

Создатель языка Кларк Эванс изначально задекларировал цели к которым стремится YAML:

1. Быть понятным человеку
1. Поддерживать структуры данных, родственные для языков программирования
1. Быть переносимым между языками программирования
1. Использовать цельную модель данных для поддержки обычного инструментария
1. Поддерживать потоковую обработку
1. Быть выразительным и расширяемым
1. Быть легким в реализации и использовании

# Взаимодействие в Python c YAML форматом

```
import yaml

with open('employees.yaml') as f:
    employees = yaml.safe_load(f)

```

```
import yaml

employees = [
    {'martin': {'job': 'Developer',
                'name': "Martin D'vloper",
                'skills': ['python', 'perl', 'pascal']
                }
     },
    {'tabitha': {'job': 'Developer',
                 'name': 'Tabitha Bitumen',
                 'skills': ['lisp', 'fortran', 'erlang']
                 }
     }
]

with open('employees.yaml', 'w') as f:
    yaml.dump(employees, f)


```

# Сериализация ExplainerDashboard

### Сгенерируем дашборд.  generate_dashboard.py:
``` python
from sklearn.ensemble import RandomForestClassifier

from explainerdashboard import *
from explainerdashboard.datasets import *

X_train, y_train, X_test, y_test = titanic_survive()
model = RandomForestClassifier(n_estimators=50, max_depth=5).fit(X_train, y_train)

explainer = ClassifierExplainer(model, X_test, y_test,
                                cats=["Sex", 'Deck', 'Embarked'],
                                labels=['Not Survived', 'Survived'],
                                descriptions=feature_descriptions)


#В классе ExplainerDashboard уже реализован собственный метод для сериализации в YAML формат (to_yaml)

db = ExplainerDashboard(explainer)
db.to_yaml("dashboard.yaml", explainerfile="explainer.joblib", dump_explainer=True)
```
                  
### Сохраним код в файле dashboard.py

https://explainerdashboard.readthedocs.io/en/latest/deployment.html#docker-deployment

## run_dashboard.py:

``` python
from explainerdashboard import ExplainerDashboard

db = ExplainerDashboard.from_config("dashboard.yaml")
db.run(host='0.0.0.0', port=9050, use_waitress=True)

```

##Dockerfile:

```
FROM python:3.11

RUN pip install explainerdashboard

COPY generate_dashboard.py ./
COPY run_dashboard.py ./

RUN python generate_dashboard.py

EXPOSE 9050
CMD ["python", "./run_dashboard.py"]
```

## запускаем

```
$ docker build -t explainerdashboard .
$ docker run -p 9050:9050 explainerdashboard

```